## Natural Language Processing - Text Clustering - Kmeans Implementation
### Fatih KIYIKÇI
### 190709032

In [5]:
import nltk
import string
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
stop_words.add('it’s')

def tokenizer(doc: str, stop_words=[]) -> str:
    if stop_words:
        #remove stopwords
        filtered = [w for w in re.findall(r'\w+', doc) if not w in stop_words and len(w) > 2] 
        doc = " ".join(filtered)
    # remove punctuation
    remove_punct = str.maketrans('', '', string.punctuation+'–•’')
    doc = doc.translate(remove_punct)
    #lowercase the letters
    doc = doc.lower()
    #remove digits
    remove_digits = str.maketrans('', '', string.digits)
    doc = doc.translate(remove_digits)
    #tokenize
    filtered = [w for w in doc.split()]
    return filtered

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fatihkykc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import numpy as np
import time
from collections import defaultdict, Counter
# tfidfVector = np.matrix(tfidfVector)

class TfidfVectorizer():
    def __init__(self, tokenizer, stopwords):
        self.tokenizer = tokenizer
        self.stopwords = stopwords


    def getTF(self, doc: str, tokenize: bool) -> dict:
        if tokenize:
            if self.stopwords:
                doc = self.tokenizer(doc,self.stopwords)
            else:
                doc = self.tokenizer(doc)
        # Counts the number of appearances of every term.
        counter = Counter(doc)

        # Divide the number of appearances of every word to the length of the document(words)
        for word in counter:
            # counter holds tf now
            counter[word] = counter[word] / len(counter)
        return counter

    def getIDF(self, TF: dict):
        counts = defaultdict()
        IDF = defaultdict()
        for doc in TF.values():
            for word in doc:
                if word in counts:
                    counts[word] += 1
                else:
                    counts[word] = 1
        for word in counts:
            IDF[word] = np.log((1 + len(corpus)) / (1 + counts[word])) + 1
        # Create a list of unique words
        self.words = counts
        self.wordDict = sorted(counts.keys())
        return IDF, counts

    # tf-idf(word) -> tf(word) * idf(word)
    def getTFIDF(self, tfDict: dict, idfDict: dict) -> dict:
        docTFIDFDict = {}
        for word in tfDict:
            docTFIDFDict[word] = tfDict[word] * idfDict[word]
        return docTFIDFDict


    def fit(self, corpus):
        # tf-idf(word) -> tf(word) * idf(word)
        TF = {}
        for i, doc in enumerate(corpus):
            TF[i] = self.getTF(doc, tokenize=True)
        # countDict = calculateCountDict(tfDict)
        IDF, counts = self.getIDF(TF)
        self.tfidfDict = [self.getTFIDF(tf, IDF) for tf in TF.values()]
        return self

    def calculateTFIDFVector(self, doctfidf):
        # Create an empty matrix to store the tfidf values
        tfidfVector = [0.0] * len(self.wordDict)

        # For each unique term, if it is in the document, store its TF-IDF value.
        for i, word in enumerate(self.wordDict):
            if word in doctfidf:
                tfidfVector[i] = doctfidf[word]
        return tfidfVector

    def get_feature_names(self):
        return [t for t, i in sorted(self.words.items(),
                              key=itemgetter(1))]

    def getMatrix(self):
        self.tfidfMatrix = [self.calculateTFIDFVector(doctfidf) for doctfidf in self.tfidfDict]
        return np.array(self.tfidfMatrix)

In [7]:
import scipy,math

def faster_cos(vec1, vec2):
    return scipy.spatial.distance.cdist(vec1.reshape(1,-1), vec2.reshape(1,-1), 'cosine')

# def dot_product(v1, v2):
#     return sum(a * b for a, b in zip(v1, v2))


# def magnitude(vector):
#     return sqrt(dot_product(vector, vector))


# def similarity(v1, v2):
#     return dot_product(v1, v2) / (magnitude(v1) * magnitude(v2) + .00000000001)


def cos_dist(vec1, vec2):
    # cosine distance implementation, results are accurate,
    # but very slow compared to scipy library, since every vector has 61k values.
    # use "faster_cos" for large corpus
    dot_product=np.dot(vec1,vec2)
    mag1 = sum([x**2 for x in vec1])
    mag2 = sum([x**2 for x in vec2])
#     print("first {}, second  {}".format(mag1,mag2))
    magnitude_product = math.sqrt(mag1) * math.sqrt(mag2)
    if not magnitude_product:
        return 0.0
    return 1 - float(dot_product) / magnitude_product

class Kmeans:
    def __init__(self, n_clusters, max_iters, tol=0.01):
        self.k = n_clusters
        self.tol = tol
        self.max_iters = max_iters
        self.centroids = []

    def euc_distance(self, a, b):
        # euclidian distance function, like cosine distance, it is slower than scipy.
        # use Scipy for large corpus
        start_time = time.time()
        dist = [(a - b) ** 2 for a, b in zip(a, b)]
        dist = math.sqrt(sum(dist))
        elapsed_time = time.time() - start_time
        print("assigned in:", elapsed_time)
        return dist

    def init_centroids(self):
        self.centroidIndex = np.random.choice(self.m,self.k,replace=False)
        self.centroids = self.data[self.centroidIndex]
        return self.centroids

    def assign_clusters(self, centroids):
        start_time = time.time()
        clusters = [[] for i in range(self.k)]
        self.labels = []
        for idx in range(len(self.data)):
#             vec = vec.reshape(-1, 1)
#             minDist = np.argmin([self.euc_distance(vec, centroids[c].reshape(-1,1)) for c in range(len(centroids))])
            minDist = np.argmin([faster_cos(np.array(self.data[idx]), np.array(centroids[c])) for c in range(self.k)])
            clusters[int(minDist)].append(idx)
            self.labels.append(int(minDist))
        elapsed_time = time.time() - start_time
        print("assigned in:", elapsed_time)
        return clusters

    def update_centroids(self, clusters):
        centroids = np.zeros((self.k, self.n))
        for cluster_idx, cluster in enumerate(clusters):
            cluster_mean = np.mean(self.data[cluster], axis=0)
            centroids[cluster_idx] = cluster_mean
        
        # self.centroids = np.array([self.data[self.cluster_labels == i].mean(axis=0) for i in range(self.k)])
        return centroids

    def predict(self):
        return self.assign_clusters()

    def fit(self, data):
        self.data = data
        self.m, self.n = self.data.shape
        print("initilizing centroids")
        self.centroids = self.init_centroids()
        for iter in range(self.max_iters):
            print("assigning centroids")
            self.cluster_labels = self.assign_clusters(self.centroids)
            self.old_centroids = self.centroids
            print("updating centroids")
            self.centroids = self.update_centroids(self.cluster_labels)
            if self.check_convergence(self.old_centroids, self.centroids):
                break
            print("Running iteration: ", iter)
        return self

    def check_convergence(self, old_centroids, centroids):
        print("checking the convergence")
        distances = [faster_cos(np.array(centroids[c]),np.array(old_centroids[c])) for c in range(self.k)]
        return sum(distances) < self.tol # tolerance value 

In [8]:
# print(faster_cos(tf_matrix[0],tf_matrix[1]))
# print(cos_dist(tf_matrix[0], tf_matrix[1]))

In [9]:
import io, os
import re as re
import zipfile as zipfile
import string
from collections import defaultdict, Counter
mytextzip = ''
corpus = []
def readfromzip(zipname: str, n_docs: int) -> None:
    global mytextzip
    global c
    with zipfile.ZipFile(zipname) as z:
        for zipinfo in z.infolist():
            if zipinfo.filename.endswith('.txt') and re.search('raw_texts', zipinfo.filename):
                with z.open(zipinfo) as f:
                    textfile = io.TextIOWrapper(f, encoding='cp1254', newline='\r\n')
                    for line in textfile:
                        if len(line):
                            if re.search(r'([a-zA-Z]+\r\n)', line):
                                mytextzip += line.strip() + ' '
                                continue
                            mytextzip += line.strip()
                    corpus.append(mytextzip)
                    if len(corpus) >= n_docs:
                        break
                    mytextzip =''
readfromzip('30Columnists.zip',1500)


In [10]:
tfidf = TfidfVectorizer(tokenizer=tokenizer, stopwords=stop_words)
tfidf = tfidf.fit(corpus)
tf_matrix = tfidf.getMatrix()

In [11]:
k_means = Kmeans(n_clusters=15, max_iters=300)

In [12]:

k_means.fit(tf_matrix)

initilizing centroids
assigning centroids
assigned in: 2.3849735260009766
updating centroids
checking the convergence
Running iteration:  0
assigning centroids
assigned in: 2.39237117767334
updating centroids
checking the convergence
Running iteration:  1
assigning centroids
assigned in: 2.348787546157837
updating centroids
checking the convergence
Running iteration:  2
assigning centroids
assigned in: 2.398411989212036
updating centroids
checking the convergence
Running iteration:  3
assigning centroids
assigned in: 2.408198118209839
updating centroids
checking the convergence
Running iteration:  4
assigning centroids
assigned in: 2.330413579940796
updating centroids
checking the convergence
Running iteration:  5
assigning centroids
assigned in: 2.3431079387664795
updating centroids
checking the convergence
Running iteration:  6
assigning centroids
assigned in: 2.329502820968628
updating centroids
checking the convergence
Running iteration:  7
assigning centroids
assigned in: 2.343080

In [16]:
# text = {}
# for i,cluster in enumerate(k_means.labels):
#     doc = corpus[i]
#     if cluster not in text.keys():
#         text[cluster] = doc
#     else:
#         text[cluster] += doc

In [17]:
# keywords = {}
# counts = {}
# for cluster in range(k_means.k):
# #     word_sent = word_tokenize(text[cluster].lower())
# #     word_sent = [word for word in word_sent if word not in stop_words]
#     word_sent = tokenizer(text[cluster], stop_words= stop_words)
#     freq = FreqDist(word_sent)
#     keywords[cluster] = nlargest(100,freq, key=freq.get)
#     counts[cluster]=freq

In [18]:
# unique_keys={}
# for cluster in range(k_means.k):
#     other_clusters = list(set(range(k_means.k))-set([cluster]))
#     keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
#     unique=set(keywords[cluster])-keys_other_clusters
#     unique_keys[cluster]=nlargest(10,unique, key=counts[cluster].get)

In [19]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tokenizer, stop_words=stop_words)
tfidf_matrix = tfidf.fit_transform(corpus)

/home/fatihkykc/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [20]:
km = KMeans(n_clusters=15)
km.fit(tfidf_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=15, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

## Display the most frequent words from each cluster

In [21]:
from nltk.probability import FreqDist
from heapq import nlargest
from nltk.tokenize import sent_tokenize, word_tokenize

In [22]:
def clusterWords(labels, cluster_centers,heuristic: bool):
    text = {}
    if not heuristic:
        cluster_centers = len(cluster_centers)
    for i,cluster in enumerate(labels):
        doc = corpus[i]
        if cluster not in text.keys():
            text[cluster] = doc
        else:
            text[cluster] += doc
    keywords = {}
    counts = {}
    for cluster in range(cluster_centers):
        word_sent = tokenizer(text[cluster], stop_words= stop_words)
        freq = FreqDist(word_sent)
        keywords[cluster] = nlargest(100,freq, key=freq.get)
        counts[cluster]=freq
    unique_keys={}
    for cluster in range(cluster_centers):
        other_clusters = list(set(range(cluster_centers))-set([cluster]))
        try:
            keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
        except:
            keys_other_clusters=set(keywords[other_clusters[0]])
        unique=set(keywords[cluster])-keys_other_clusters
        unique_keys[cluster]=nlargest(10,unique, key=counts[cluster].get)
    return unique_keys

In [23]:
sklearnRes = clusterWords(km.labels_, km.cluster_centers_,heuristic=False)

In [24]:
HeuristicRes = clusterWords(k_means.labels, k_means.k, heuristic=True)

In [25]:
sklearnRes

{0: ['snp',
  'labour',
  'government',
  'elections',
  'scottish',
  'says',
  'tax',
  'election',
  'council',
  'parliament'],
 1: ['study',
  'risk',
  'patients',
  'women',
  'disease',
  'found',
  'heart',
  'studies',
  'blood',
  'health'],
 2: ['league',
  'football',
  'club',
  'united',
  'west',
  'fans',
  'ham',
  'premier',
  'season',
  'cup'],
 3: ['bank',
  'bn',
  'barclays',
  'banking',
  'hbos',
  'capital',
  'banks',
  'shares',
  'lloyds',
  'investors'],
 4: ['independence',
  'powers',
  'brown',
  'referendum',
  'power',
  'policy',
  'devolution',
  'gordon',
  'scots',
  'glasgow'],
 5: ['insurance',
  'plans',
  'plan',
  'pay',
  'company',
  'coverage',
  'medicare',
  'companies',
  'doctor',
  'cost'],
 6: ['obama',
  'mccain',
  'president',
  'barack',
  'hillary',
  'clinton',
  'state',
  'campaign',
  'world',
  'senate'],
 7: ['brown',
  'gordon',
  'prime',
  'cameron',
  'david',
  'tory',
  'that',
  'blair',
  'cabinet',
  'britain'],


In [26]:
HeuristicRes

{0: ['united',
  'liverpool',
  'chelsea',
  'league',
  'team',
  'season',
  'ferguson',
  'last',
  'manchester',
  'mourinho'],
 1: ['kids',
  'family',
  'school',
  'child',
  'fitness',
  'parents',
  'workout',
  'often',
  'muscles',
  'training'],
 2: ['study',
  'risk',
  'disease',
  'blood',
  'studies',
  'found',
  'vitamin',
  'brain',
  'researchers',
  'high'],
 3: ['alcohol',
  'drinking',
  'drink',
  'women',
  'found',
  'per',
  'blood',
  'study',
  'pubs',
  'cent'],
 4: ['obama',
  'new',
  'and',
  'president',
  'mccain',
  'public',
  'state',
  'say',
  'management',
  'house'],
 5: ['baseball',
  'new',
  'series',
  'games',
  'mets',
  'york',
  'yankees',
  'sox',
  'run',
  'since'],
 6: ['patients',
  'medicare',
  'drugs',
  'drug',
  'plan',
  'treatment',
  'plans',
  'part',
  'effects',
  'care'],
 7: ['city',
  'newcastle',
  'new',
  'kinnear',
  'hughes',
  'and',
  'ashley',
  'million',
  'joey',
  'robinho'],
 8: ['bank',
  'government',
 